# Demo caching

This notebook shows how caching of daily results is organised. First we show the low-level approach, then a high-level function is used.

## Low-level approach

In [ ]:
import pandas as pd
from opengrid.library import misc
from opengrid.library import houseprint
from opengrid.library import caching
from opengrid.library import analysis
import charts

In [ ]:
hp = houseprint.Houseprint()
hp.sync_tmpos()

We demonstrate the caching for the minimal daily water consumption (should be close to zero unless there is a water leak). We start from the water consumption of last week.

In [ ]:
start = pd.Timestamp('now') - pd.Timedelta(weeks=1)
df_water = hp.get_data(sensortype='water', head=start).diff()
df_water.info()

We use the method *daily_min()* from the analysis module to obtain a dataframe with daily minima for each sensor.

In [ ]:
daily_min = analysis.daily_min(df_water)
daily_min.info()

We create a cache object by specifying what we like to store in this object.  The cached data is saved as a single csv per sensor in a folder specified in the opengrid.cfg.  Add a this to your opengrid.cfg

    [data]
    folder: path_to_folder

In [ ]:
cache1 = caching.Cache(result='water_daily_min')

In [ ]:
df_cache = cache1.get(sensors=[s.key for s in hp.get_sensors(sensortype='water')])

In [ ]:
cache1.update(daily_min)

Now we can get the daily water minima from the cache directly. Pass a *start* or *end* date to limit the returned dataframe.

In [ ]:
sensors = [s.key for s in hp.get_sensors(sensortype='water')] # sensor ids
charts.plot(cache1.get(sensors=sensors, start=start, end=None), show='inline', stock=True)

## A high-level cache function

The caching of daily results is very similar for all kinds of results.  Therefore, a high-level function is defined that can be parametrised to cache a lot of different things.

In [ ]:
import pandas as pd
from opengrid.library import misc
from opengrid.library import houseprint
from opengrid.library import caching
from opengrid.library import analysis
import charts   

In [ ]:
hp = houseprint.Houseprint()
#hp.sync_tmpos()

In [ ]:
sensors = hp.get_sensors(sensortype='water')
caching.cache(hp=hp, sensors=sensors, function='daily_min', resultname='water_daily_min')

In [ ]:
cache = caching.Cache('water_daily_min')
daily_min = cache.get(sensors = [s.key for s in sensors], start = '20151201')
charts.plot(daily_min, stock=True, show='inline')